In [1]:
%%writefile ../bin/csv-tables.py
#!/usr/bin/env python3

"""
Panflute filter to parse CSV in fenced YAML code blocks

5 metadata keys are recognized:

-   title: the caption of the table. If omitted, no title will be inserted.
-   has-header: If true, has a header row. default: true
-   column-width: a list of relative width corresponding to the width of each columns.
    default: auto calculate from the length of line in a (potentially multiline) cell.
-   table-width: the relative width of the table (comparing to, say, \linewidth).
    default: 1.0
-   alignment: a string of characters among L,R,C,D, case-insensitive,
    corresponds to Left-aligned, Right-aligned, Center-aligned, Default-aligned respectively.
    e.g. LCRD for a table with 4 columns
    default: DDD...

When the metadata keys is invalid, the default will be used instead.

e.g.

```markdown
~~~csv
title: "*Great* Title"
has-header: False
column-width:
  - 0.1
  - 0.2
  - 0.3
  - 0.4
alignment: LRC
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~
```
"""

import io
import csv
import panflute

def fenced_csv(options, data, element, doc):
    # read csv and convert to panflute table representation
    with io.StringIO(data) as f:
        reader = list(csv.reader(f))
        body = []
        for row in reader:
            cells = [panflute.TableCell(*panflute.convert_text(x)) for x in row]
            body.append(panflute.TableRow(*cells))
        # get no of columns for header
        noOfColumn = len(reader[0])
    # read YAML metadata
    try:
        caption = str(options.get('title'))
        column_width = options.get('column-width')
        table_width = options.get('table-width',1.0)
        alignment = str(options.get('alignment'))
        has_header = options.get('has-header',True)
    except AttributeError:
        caption = None
        column_width = None
        table_width = 1.0
        alignment = None
        has_header = True
    # check if YAML is valid
    try:
        column_width = [(float(x) if float(x)>0 else 0) for x in column_width]
    except (TypeError, ValueError):
        column_width = None
    try:
        table_width = float(table_width)
    except (TypeError, ValueError):
        column_width = None
    if not isinstance(has_header, bool):
        has_header = True
    # get caption
    if caption != None:
        caption = panflute.convert_text(caption)[0].content
    # get column_width
    if column_width == None:
        column_width_abs = [max([max([len(line) for line in row[i].split("\n")]) for row in list(reader)]) for i in range(noOfColumn)]
        column_width_tot = sum(column_width_abs)
        column_width = [column_width_abs[i]/column_width_tot*table_width for i in range(noOfColumn)]
    # get alignment
    if alignment != None:
        parsed_alignment = []
        for i in range(noOfColumn):
            try:
                if alignment[i].lower() == "l":
                    parsed_alignment.append("AlignLeft")
                elif alignment[i].lower() == "c":
                    parsed_alignment.append("AlignCenter")
                elif alignment[i].lower() == "r":
                    parsed_alignment.append("AlignRight")
                else:
                    parsed_alignment.append("AlignDefault")
            except IndexError:
                for i in range(noOfColumn-len(parsed_alignment)):
                    parsed_alignment.append("AlignDefault")
        alignment = parsed_alignment
    # finalize table according to metadata
    header = body.pop(0) if has_header else None # panflute.TableRow(*[panflute.TableCell() for i in range(noOfColumn)]) # for panflute < 1.4.3
    table = panflute.Table(*body, header=header, caption=caption, width=column_width, alignment=alignment)
    return table

# We'll only run this for CodeBlock elements of class 'csv'
if __name__ == '__main__':
    panflute.toJSONFilter(panflute.yaml_filter, tag='csv', function=fenced_csv)

Overwriting ../bin/csv-tables.py


In [2]:
!chmod +x ../bin/csv-tables.py

In [3]:
%%writefile comparison.md
~~~csv
table-width: 1.3
---
,pandoc-csv2table,pandoc-placetable,panflute example,my proposal
type,type=simple|multiline|grid|pipe,,,
header,header=yes|no,header=yes|no,has-header: True|False,header: True|False
caption,caption,caption,title,caption
source,source,file,source,source
aligns,aligns=LRCD,aligns=LRCD,,alignment: LRCD
width,,"widths=""0.5 0.2 0.3""",,"column-width: [0.5, 0.2, 0.3]"
,,inlinemarkdown,,markdown: True|False
,,delimiter,,
,,quotechar,,
,,id (wrapped by div),,
~~~

Overwriting comparison.md


In [4]:
print("JSON", "Comparison", "="*80)
!pandoc comparison.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Comparison", "="*80)
!pandoc --filter=../bin/csv-tables.py comparison.md -t native
print("\n", "Markdown", "Comparison", "="*80)
!pandoc --filter=../bin/csv-tables.py comparison.md -t markdown

JSON Comparison ================================================================================
{"pandoc-api-version":[1,17,0,4],"meta":{},"blocks":[{"t":"Table","c":[[{"t":"Str","c":"None"}],[{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"}],[0.0834862385321101,0.3697247706422019,0.23853211009174313,0.26238532110091745,0.34587155963302757],[[],[{"t":"Para","c":[{"t":"Str","c":"pandoc-csv2table"}]}],[{"t":"Para","c":[{"t":"Str","c":"pandoc-placetable"}]}],[{"t":"Para","c":[{"t":"Str","c":"panflute"},{"t":"Space"},{"t":"Str","c":"example"}]}],[{"t":"Para","c":[{"t":"Str","c":"my"},{"t":"Space"},{"t":"Str","c":"proposal"}]}]],[[[{"t":"Para","c":[{"t":"Str","c":"type"}]}],[{"t":"Para","c":[{"t":"Str","c":"type=simple|multiline|grid|pipe"}]}],[],[],[]],[[{"t":"Para","c":[{"t":"Str","c":"header"}]}],[{"t":"Para","c":[{"t":"Str","c":"header=yes|no"}]}],[{"t":"Para","c":[{"t":"Str","c":"header=yes|no"}]}],[{"t":"Para","c":[{"t":"Str","c

In [5]:
%%writefile test-simple.md
~~~csv
table-width: 1.2
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


Overwriting test-simple.md


In [6]:
print("JSON", "Simple", "="*80)
!pandoc test-simple.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Simple", "="*80)
!pandoc --filter=../bin/csv-tables.py test-simple.md -t native
print("\n", "Markdown", "Simple", "="*80)
!pandoc --filter=../bin/csv-tables.py test-simple.md -t markdown

JSON Simple ================================================================================
{"pandoc-api-version":[1,17,0,4],"meta":{},"blocks":[{"t":"Table","c":[[{"t":"Str","c":"None"}],[{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"}],[0.22499999999999998,0.16874999999999998,0.22499999999999998,0.5812499999999999],[[{"t":"Para","c":[{"t":"Strong","c":[{"t":"Emph","c":[{"t":"Str","c":"Fruit"}]}]}]}],[{"t":"Para","c":[{"t":"Strikeout","c":[{"t":"Str","c":"Price"}]}]}],[{"t":"Para","c":[{"t":"Emph","c":[{"t":"Str","c":"Number"}]}]}],[{"t":"Para","c":[{"t":"Code","c":[["",[],[]],"Advantages"]}]}]],[[[{"t":"Para","c":[{"t":"Emph","c":[{"t":"Str","c":"Bananas"},{"t":"Subscript","c":[{"t":"Str","c":"1"}]}]}]}],[{"t":"Para","c":[{"t":"Str","c":"$1.34"}]}],[{"t":"Para","c":[{"t":"Str","c":"12"},{"t":"Subscript","c":[{"t":"Str","c":"units"}]}]}],[{"t":"Para","c":[{"t":"Str","c":"Benefits"},{"t":"Space"},{"t":"Str","c":"of"},{"t":"Space"},{"t":"Str","c":"ea

In [7]:
%%writefile test-full.md
~~~csv
title: "*Great* Title"
has-header: False
column-width:
  - 0.1
  - 0.2
  - 0.3
  - 0.4
alignment: LRC
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


Overwriting test-full.md


In [8]:
print("JSON", "Full", "="*80)
!pandoc test-full.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Full", "="*80)
!pandoc --filter=../bin/csv-tables.py test-full.md -t native
print("\n", "Markdown", "Full", "="*80)
!pandoc --filter=../bin/csv-tables.py test-full.md -t markdown

JSON Full ================================================================================
{"pandoc-api-version":[1,17,0,4],"meta":{},"blocks":[{"t":"Table","c":[[{"t":"Emph","c":[{"t":"Str","c":"Great"}]},{"t":"Space"},{"t":"Str","c":"Title"}],[{"t":"AlignLeft"},{"t":"AlignRight"},{"t":"AlignCenter"},{"t":"AlignDefault"}],[0.1,0.2,0.3,0.4],[],[[[{"t":"Para","c":[{"t":"Strong","c":[{"t":"Emph","c":[{"t":"Str","c":"Fruit"}]}]}]}],[{"t":"Para","c":[{"t":"Strikeout","c":[{"t":"Str","c":"Price"}]}]}],[{"t":"Para","c":[{"t":"Emph","c":[{"t":"Str","c":"Number"}]}]}],[{"t":"Para","c":[{"t":"Code","c":[["",[],[]],"Advantages"]}]}]],[[{"t":"Para","c":[{"t":"Emph","c":[{"t":"Str","c":"Bananas"},{"t":"Subscript","c":[{"t":"Str","c":"1"}]}]}]}],[{"t":"Para","c":[{"t":"Str","c":"$1.34"}]}],[{"t":"Para","c":[{"t":"Str","c":"12"},{"t":"Subscript","c":[{"t":"Str","c":"units"}]}]}],[{"t":"Para","c":[{"t":"Str","c":"Benefits"},{"t":"Space"},{"t":"Str","c":"of"},{"t":"Space"},{"t":"Str","c":"eating"},{"t

In [9]:
%%writefile test-type.md
~~~csv
title: 0.1
has-header: IDK
column-width:
- -0.1
- -0.2
- -0.3
- -0.4
alignment: 0.1
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


Overwriting test-type.md


In [10]:
print("JSON", "Type", "="*80)
!pandoc test-type.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Type", "="*80)
!pandoc --filter=../bin/csv-tables.py test-type.md -t native
print("\n", "Markdown", "Type", "="*80)
!pandoc --filter=../bin/csv-tables.py test-type.md -t markdown

JSON Type ================================================================================
{"pandoc-api-version":[1,17,0,4],"meta":{},"blocks":[{"t":"Table","c":[[{"t":"Str","c":"0.1"}],[{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"}],[0,0,0,0],[[{"t":"Para","c":[{"t":"Strong","c":[{"t":"Emph","c":[{"t":"Str","c":"Fruit"}]}]}]}],[{"t":"Para","c":[{"t":"Strikeout","c":[{"t":"Str","c":"Price"}]}]}],[{"t":"Para","c":[{"t":"Emph","c":[{"t":"Str","c":"Number"}]}]}],[{"t":"Para","c":[{"t":"Code","c":[["",[],[]],"Advantages"]}]}]],[[[{"t":"Para","c":[{"t":"Emph","c":[{"t":"Str","c":"Bananas"},{"t":"Subscript","c":[{"t":"Str","c":"1"}]}]}]}],[{"t":"Para","c":[{"t":"Str","c":"$1.34"}]}],[{"t":"Para","c":[{"t":"Str","c":"12"},{"t":"Subscript","c":[{"t":"Str","c":"units"}]}]}],[{"t":"Para","c":[{"t":"Str","c":"Benefits"},{"t":"Space"},{"t":"Str","c":"of"},{"t":"Space"},{"t":"Str","c":"eating"},{"t":"Space"},{"t":"Str","c":"bananas"},{"t":"SoftBreak"},{"t":"Str

In [11]:
%%writefile test-include.md
~~~csv
title: "*Great* Title"
has-header: False
column-width: 0.1, 0.2, 0.3, 0.4
alignment: AlignLeft, AlignRight, AlignCenter, AlignDefault
source: ../source/grid.csv
~~~

Overwriting test-include.md


In [12]:
print("JSON", "Include", "="*80)
!pandoc test-include.md -t json | ../bin/csv-tables.py
print("\n", "Native", "Include", "="*80)
!pandoc --filter=../bin/csv-tables.py test-include.md -t native
print("\n", "Markdown", "Include", "="*80)
!pandoc --filter=../bin/csv-tables.py test-include.md -t markdown

JSON Include ================================================================================
Traceback (most recent call last):
  File "../bin/csv-tables.py", line 121, in <module>
    panflute.toJSONFilter(panflute.yaml_filter, tag='csv', function=fenced_csv)
  File "/usr/local/lib/python3.5/site-packages/panflute/io.py", line 179, in toJSONFilter
    return run_filter(*args, **kwargs)
  File "/usr/local/lib/python3.5/site-packages/panflute/io.py", line 249, in run_filter
    return run_filters([action], *args, **kwargs)
  File "/usr/local/lib/python3.5/site-packages/panflute/io.py", line 231, in run_filters
    doc = doc.walk(action, doc)
  File "/usr/local/lib/python3.5/site-packages/panflute/base.py", line 248, in walk
    ans = list(chain.from_iterable(ans))
  File "/usr/local/lib/python3.5/site-packages/panflute/base.py", line 246, in <genexpr>
    ans = ((item,) if type(item) != list else item for item in ans)
  File "/usr/local/lib/python3.5/site-packages/panflute/base.py", li